In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
import torch
import json
import pandas as pd
import numpy as np
import random
import nltk
import re
from collections import defaultdict
from utils import get_qa_pair, add_span, get_word_dict, bioclean, text_to_list, squad_to_bioasq_format, flatten_span_list, formalize_data
from model import BaselineModel

BiLSTM(Q)
BiLSTM(C)
softmax([Ocs, Oce]WOq)
cross_entropy

In [2]:
with open('./data/BioASQ-trainingDataset5b.txt', encoding='utf8') as f:
    data = json.load(f)

In [25]:
with open('./data/dev-v1.1.json', encoding='utf8') as f:
    squad_data = json.load(f)

In [27]:
squad_qa_pair = squad_to_bioasq_format(squad_data)
print ('adding spans into the qa_pair')
squad_qa_pair = add_span(squad_qa_pair)
# (q, a, t, s, span_lst)
squad_qa_pair = text_to_list(squad_qa_pair)
# # Debugging span extraction
# for (snippet, spans) in zip(qa_pair[0][3], qa_pair[0][4]):
#     print (spans)
#     print (snippet)
#     for (start, end) in spans:
#         print (snippet[start:end + 1])
        
bioword_dict = get_word_dict(squad_qa_pair, bioclean, bioword_dict)
squad_train, squad_test = train_test_split(squad_qa_pair, test_size=0.2, random_state=32)
squad_train, squad_validate = train_test_split(squad_train, test_size=0.2, random_state=32)
squad_train_flat = formalize_data(flatten_span_list(squad_train), bioword_dict)
squad_test_flat = formalize_data(flatten_span_list(squad_test), bioword_dict)
squad_validate_flat = formalize_data(flatten_span_list(squad_validate), bioword_dict)

questions count: 10570
adding spans into the qa_pair
10566/10570=0.9996215704824977 question have exact answer in span
vocabulary from 31336 to 31336


In [ ]:
import time
torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

epoch_num = 50
hidden_dim = 100
model = BaselineModel(hidden_dim, 200, len(bioword_dict))
model.cuda()
model.load_embed_bioasq(bioword_dict, './data/vectors.txt', './data/types.txt')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

dev_acc = []
dev_loss = []
for sample in squad_validate_flat:
    q = sample.q_words.numpy().tolist()
    context = sample.d_words.numpy().tolist()
    span = sample.span
    index = len(context)*span[0] + span[1]
    pred = model(q, context)

    loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))

    pred_index = np.argmax(pred[0].data.cpu())
    dev_acc.append(pred_index == index)
    dev_loss.append(loss.data.cpu()[0])
print ('validation acc:', np.mean(dev_acc))
print ('validation loss:', np.mean(dev_loss))

for epoch in range(epoch_num):
    print ('epoch:', epoch)
    model.train()
    #random.shuffle(squad_train_flat)
    
    train_acc = []
    train_loss = []
    for sample in squad_train_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        
        optimizer.zero_grad()
        pred = model(q, context)
        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        loss.backward()
        
        pred_index = np.argmax(pred[0].data.cpu())
        train_acc.append(pred_index == index)
        train_loss.append(loss.data.cpu()[0])
        optimizer.step()
    
    print ('train loss:', np.mean(train_loss))
    print ('train acc:', np.mean(train_acc))
    
    model.eval()
    dev_acc = []
    dev_loss = []
    for sample in squad_validate_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        
        pred_span = (pred_index//len(context), pred_index%len(context))
        
        dev_acc.append(pred_index == index)
        dev_loss.append(loss.data.cpu()[0])
    
    print ('validation acc:', np.mean(dev_acc))
    print ('validation loss:', np.mean(dev_loss))
    
    model.eval()
    test_acc = []
    test_loss = []
    for sample in squad_test_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        test_acc.append(pred_index == index)
        test_loss.append(loss.data.cpu()[0])
    print ('test acc:', np.mean(test_acc))
    print ('test loss:', np.mean(test_loss))

loading embeddings...


In [41]:
len(squad_train), len(squad_test), len(squad_validate)

(6761, 2114, 1691)

In [42]:
len(squad_train_flat), len(squad_test_flat), len(squad_validate_flat)

(14074, 4296, 3602)

In [4]:
print ('getting qa pair from data')
qa_pair = get_qa_pair(data)
#qa_pair = squad_to_bioasq_format(squad_data)

print ('adding spans into the qa_pair')
qa_pair = add_span(qa_pair)
# (q, a, t, s, span_lst)
qa_pair = text_to_list(qa_pair)
# # Debugging span extraction
# for (snippet, spans) in zip(qa_pair[0][3], qa_pair[0][4]):
#     print (spans)
#     print (snippet)
#     for (start, end) in spans:
#         print (snippet[start:end + 1])
        
bioword_dict = get_word_dict(qa_pair, bioclean)

getting qa pair from data
duplicate snippet count: 3587
questions count: 484
adding spans into the qa_pair
330/484=0.6818181818181818 question have exact answer in span
vocabulary from 2 to 8882


In [5]:
from sklearn.model_selection import train_test_split
np.random.seed(1)

train, test = train_test_split(qa_pair, test_size=0.2, random_state=32)
train, validate = train_test_split(train, test_size=0.2, random_state=32)
train_flat = formalize_data(flatten_span_list(train), bioword_dict)
test_flat = formalize_data(flatten_span_list(test), bioword_dict)
validate_flat = formalize_data(flatten_span_list(validate), bioword_dict)

In [6]:
len(train), len(test), len(validate)

(211, 66, 53)

In [7]:
len(train_flat), len(test_flat), len(validate_flat)

(2298, 626, 347)

In [20]:
import time
torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

epoch_num = 50
hidden_dim = 100
model = BaselineModel(hidden_dim, 200, len(bioword_dict))
model.cuda()
model.load_embed_bioasq(bioword_dict, './data/vectors.txt', './data/types.txt')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

dev_acc = []
dev_loss = []
for sample in validate_flat:
    q = sample.q_words.numpy().tolist()
    context = sample.d_words.numpy().tolist()
    span = sample.span
    index = len(context)*span[0] + span[1]
    pred = model(q, context)

    loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))

    pred_index = np.argmax(pred[0].data.cpu())
    dev_acc.append(pred_index == index)
    dev_loss.append(loss.data.cpu()[0])
print ('validation acc:', np.mean(dev_acc))
print ('validation loss:', np.mean(dev_loss))

for epoch in range(epoch_num):
    print ('epoch:', epoch)
    model.train()
    #random.shuffle(train_flat)
    
    train_acc = []
    train_loss = []
    for sample in train_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        
        optimizer.zero_grad()
        pred = model(q, context)
        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        loss.backward()
        
        pred_index = np.argmax(pred[0].data.cpu())
        train_acc.append(pred_index == index)
        train_loss.append(loss.data.cpu()[0])
        optimizer.step()
    
    print ('train loss:', np.mean(train_loss))
    print ('train acc:', np.mean(train_acc))
    
    model.eval()
    dev_acc = []
    dev_loss = []
    for sample in validate_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        
        pred_span = (pred_index//len(context), pred_index%len(context))
        print (span, pred_span)
        print (sample.snippet_text.split(' ')[span[0]:span[1] + 1])
        print (sample.snippet_text.split(' ')[pred_span[0]:pred_span[1] + 1])
        
        dev_acc.append(pred_index == index)
        dev_loss.append(loss.data.cpu()[0])
    raise
    print ('validation acc:', np.mean(dev_acc))
    print ('validation loss:', np.mean(dev_loss))
    
    model.eval()
    test_acc = []
    test_loss = []
    for sample in test_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        test_acc.append(pred_index == index)
        test_loss.append(loss.data.cpu()[0])
    print ('test acc:', np.mean(test_acc))
    print ('test loss:', np.mean(test_loss))

loading embeddings...
load 8411/8882 embeddings
validation acc: 0.0115273775216
validation loss: 6.84873
epoch: 0
train loss: 5.57219
train acc: 0.165796344648

 10
 10
[torch.LongTensor of size 2]
 (10, 10)
['zinc-dependent']
['zinc-dependent']

 14
 14
[torch.LongTensor of size 2]
 (14, 14)
['zinc']
['zinc']

 14
 14
[torch.LongTensor of size 2]
 (0, 5)
['Meningiomas']
['Simpson', 'Grade', 'I-III', 'Resection', 'of', 'Spinal']

 13
 13
[torch.LongTensor of size 2]
 (13, 13)
['meningiomas']
['meningiomas']

 13
 13
[torch.LongTensor of size 2]
 (56, 56)
['meningioma']
['benign']

 62
 62
[torch.LongTensor of size 2]
 (56, 56)
['meningioma']
['benign']

 9
 9
[torch.LongTensor of size 2]
 (9, 9)
['meningiomas']
['meningiomas']

 76
 76
[torch.LongTensor of size 2]
 (76, 76)
['meningiomas']
['meningiomas']

 93
 93
[torch.LongTensor of size 2]
 (76, 76)
['meningiomas']
['meningiomas']

 17
 17
[torch.LongTensor of size 2]
 (17, 17)
['meningiomas']
['meningiomas']

 159
 159
[torch.LongT

 (12, 12)
['Riociguat']
['guanylate']

 6
 6
[torch.LongTensor of size 2]
 (15, 15)
['riociguat']
['hypertension']

 16
 16
[torch.LongTensor of size 2]
 (22, 22)
['l-DOPA']
['benserazide']

 2
 2
[torch.LongTensor of size 2]
 (10, 10)
['L-Dopa']
['benserazide']

 4
 4
[torch.LongTensor of size 2]
 (69, 69)
['l-DOPA-primed']
['benserazide']

 64
 64
[torch.LongTensor of size 2]
 (69, 69)
['l-DOPA']
['benserazide']

 0
 0
[torch.LongTensor of size 2]
 (11, 11)
['L-DOPA']
['benserazide']

 21
 21
[torch.LongTensor of size 2]
 (11, 11)
['L-DOPA-induced']
['benserazide']

 1
 1
[torch.LongTensor of size 2]
 (6, 6)
['L-DOPA-treated']
['receptor']

 64
 64
[torch.LongTensor of size 2]
 (6, 6)
['L-DOPA']
['receptor']

 18
 18
[torch.LongTensor of size 2]
 (33, 33)
['L-DOPA']
['benserazide']

 49
 49
[torch.LongTensor of size 2]
 (33, 33)
['L-DOPA']
['benserazide']

 8
 8
[torch.LongTensor of size 2]
 (18, 18)
['L-DOPA']
['benserazide']

 25
 25
[torch.LongTensor of size 2]
 (18, 18)
['L-DOPA/


 2
 2
[torch.LongTensor of size 2]
 (2, 2)
['calsequestrin']
['calsequestrin']

 4
 4
[torch.LongTensor of size 2]
 (2, 2)
['CASQ2']
['calsequestrin']

 0
 0
[torch.LongTensor of size 2]
 (4, 4)
['Calsequestrin']
['calcium-binding']

 0
 0
[torch.LongTensor of size 2]
 (3, 3)
['Calsequestrin']
['calcium-binding']

 0
 0
[torch.LongTensor of size 2]
 (23, 23)
['Calsequestrin']
['calcium-binding']

 2
 2
[torch.LongTensor of size 2]
 (23, 23)
['CASQ']
['calcium-binding']

 11
 11
[torch.LongTensor of size 2]
 (8, 8)
['calsequestrin']
['calcium-binding']

 0
 0
[torch.LongTensor of size 2]
 (35, 35)
['Calsequestrin']
['triadin']

 0
 0
[torch.LongTensor of size 2]
 (20, 20)
['Calsequestrin']
['ryanodine']

 7
 9
[torch.LongTensor of size 2]
 (21, 21)
['secretory', 'phospholipase', 'A2']
['syndrome']

 13
 15
[torch.LongTensor of size 2]
 (13, 13)
['secretory', 'phospholipase', 'A2']
['secretory']

 15
 15
[torch.LongTensor of size 2]
 (1, 6)
['S-phase']
['chromatid', 'cohesion', 'is', 'm


 7
 7
[torch.LongTensor of size 2]
 (38, 38)
['naloxone']
['naloxone']

 38
 38
[torch.LongTensor of size 2]
 (38, 38)
['naloxone']
['naloxone']

 22
 22
[torch.LongTensor of size 2]
 (22, 22)
['naloxone']
['naloxone']

  0
 16
[torch.LongTensor of size 2]
 (14, 14)
['The', 'beet', 'Y', 'locus', 'encodes', 'an', 'anthocyanin', 'MYB-like', 'protein', 'that', 'activates', 'the', 'betalain', 'red', 'pigment', 'pathway', '.']
['pigment']

 20
 21
[torch.LongTensor of size 2]
 (12, 12)
['BRCA1', 'dysfunction']
['triple-negative']

 0
 1
[torch.LongTensor of size 2]
 (0, 0)
['autosomal', 'dominant']
['autosomal']

 9
 9
[torch.LongTensor of size 2]
 (9, 9)
['antiparallel']
['antiparallel']

 0
 0
[torch.LongTensor of size 2]
 (1, 1)
['antiparallel']
['spectrin']

 28
 28
[torch.LongTensor of size 2]
 (1, 13)
['antiparallel']
['erythroid', 'alpha-spectrin', 'repeats', '13', 'and', '14', '(', 'HEalpha13,14', ')', 'and', 'human', 'erythroid', 'beta-spectrin']

 26
 26
[torch.LongTensor of size

RuntimeError: No active exception to reraise

In [21]:
import time
torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

epoch_num = 50
hidden_dim = 100
model = BaselineModel(hidden_dim, 200, len(bioword_dict))
model.cuda()
model.load_embed_bioasq(bioword_dict, './data/vectors.txt', './data/types.txt')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

dev_acc = []
dev_loss = []
for sample in validate_flat:
    q = sample.q_words.numpy().tolist()
    context = sample.d_words.numpy().tolist()
    span = sample.span
    index = len(context)*span[0] + span[1]
    pred = model(q, context)

    loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))

    pred_index = np.argmax(pred[0].data.cpu())
    dev_acc.append(pred_index == index)
    dev_loss.append(loss.data.cpu()[0])
print ('validation acc:', np.mean(dev_acc))
print ('validation loss:', np.mean(dev_loss))

for epoch in range(epoch_num):
    print ('epoch:', epoch)
    model.train()
    #random.shuffle(train_flat)
    
    train_acc = []
    train_loss = []
    for sample in train_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        
        optimizer.zero_grad()
        pred = model(q, context)
        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        loss.backward()
        
        pred_index = np.argmax(pred[0].data.cpu())
        train_acc.append(pred_index == index)
        train_loss.append(loss.data.cpu()[0])
        optimizer.step()
    
    print ('train loss:', np.mean(train_loss))
    print ('train acc:', np.mean(train_acc))
    
    model.eval()
    dev_acc = []
    dev_loss = []
    for sample in validate_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        
        pred_span = (pred_index//len(context), pred_index%len(context))
        
        dev_acc.append(pred_index == index)
        dev_loss.append(loss.data.cpu()[0])

    print ('validation acc:', np.mean(dev_acc))
    print ('validation loss:', np.mean(dev_loss))
    
    model.eval()
    test_acc = []
    test_loss = []
    for sample in test_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        test_acc.append(pred_index == index)
        test_loss.append(loss.data.cpu()[0])
    print ('test acc:', np.mean(test_acc))
    print ('test loss:', np.mean(test_loss))

loading embeddings...
load 8411/8882 embeddings
validation acc: 0.0115273775216
validation loss: 6.84873
epoch: 0
train loss: 5.57219
train acc: 0.165796344648
validation acc: 0.161383285303
validation loss: 5.57128
test acc: 0.0958466453674
test loss: 5.78612
epoch: 1
train loss: 3.6381
train acc: 0.301566579634
validation acc: 0.219020172911
validation loss: 4.59573
test acc: 0.110223642173
test loss: 4.70016
epoch: 2
train loss: 2.56786
train acc: 0.442123585727
validation acc: 0.247838616715
validation loss: 4.3824
test acc: 0.135782747604
test loss: 4.28241
epoch: 3
train loss: 2.04023
train acc: 0.514795474326
validation acc: 0.256484149856
validation loss: 4.33294
test acc: 0.167731629393
test loss: 4.23778
epoch: 4
train loss: 1.72263
train acc: 0.557876414273
validation acc: 0.227665706052
validation loss: 4.35505
test acc: 0.170926517572
test loss: 4.2751
epoch: 5
train loss: 1.49723
train acc: 0.593124456049
validation acc: 0.233429394813
validation loss: 4.53105
test acc: 0

In [22]:
import time
torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

epoch_num = 50
hidden_dim = 50
model = BaselineModel(hidden_dim, 200, len(bioword_dict))
model.cuda()
model.load_embed_bioasq(bioword_dict, './data/vectors.txt', './data/types.txt')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

for epoch in range(epoch_num):
    print ('epoch:', epoch)
    model.train()
    #random.shuffle(train_flat)
    
    train_acc = []
    train_loss = []
    for sample in train_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        
        optimizer.zero_grad()
        pred = model(q, context)
        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        loss.backward()
        
        pred_index = np.argmax(pred[0].data.cpu())
        train_acc.append(pred_index == index)
        train_loss.append(loss.data.cpu()[0])
        optimizer.step()
    
    print ('train loss:', np.mean(train_loss))
    print ('train acc:', np.mean(train_acc))
    
    model.eval()
    dev_acc = []
    dev_loss = []
    for sample in validate_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        dev_acc.append(pred_index == index)
        dev_loss.append(loss.data.cpu()[0])
    print ('validation acc:', np.mean(dev_acc))
    print ('validation loss:', np.mean(dev_loss))
    
    model.eval()
    test_acc = []
    test_loss = []
    for sample in test_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        test_acc.append(pred_index == index)
        test_loss.append(loss.data.cpu()[0])
    print ('test acc:', np.mean(test_acc))
    print ('test loss:', np.mean(test_loss))

loading embeddings...
load 8411/8882 embeddings
epoch: 0
train loss: 5.54092
train acc: 0.152306353351
validation acc: 0.164265129683
validation loss: 5.62813
test acc: 0.102236421725
test loss: 5.89964
epoch: 1
train loss: 3.55213
train acc: 0.306788511749
validation acc: 0.233429394813
validation loss: 4.46295
test acc: 0.127795527157
test loss: 4.69669
epoch: 2
train loss: 2.57365
train acc: 0.435596170583
validation acc: 0.265129682997
validation loss: 4.22009
test acc: 0.167731629393
test loss: 4.37476
epoch: 3
train loss: 2.06253
train acc: 0.514795474326
validation acc: 0.273775216138
validation loss: 4.18694
test acc: 0.174121405751
test loss: 4.27317
epoch: 4
train loss: 1.75049
train acc: 0.558746736292
validation acc: 0.242074927954
validation loss: 4.30396
test acc: 0.17571884984
test loss: 4.33154
epoch: 5
train loss: 1.52583
train acc: 0.593124456049
validation acc: 0.230547550432
validation loss: 4.36111
test acc: 0.199680511182
test loss: 4.45737
epoch: 6
train loss: 1.

In [23]:
import time
torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

epoch_num = 50
hidden_dim = 150
model = BaselineModel(hidden_dim, 200, len(bioword_dict))
model.cuda()
model.load_embed_bioasq(bioword_dict, './data/vectors.txt', './data/types.txt')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

for epoch in range(epoch_num):
    print ('epoch:', epoch)
    model.train()
    #random.shuffle(train_flat)
    
    train_acc = []
    train_loss = []
    for sample in train_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        
        optimizer.zero_grad()
        pred = model(q, context)
        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        loss.backward()
        
        pred_index = np.argmax(pred[0].data.cpu())
        train_acc.append(pred_index == index)
        train_loss.append(loss.data.cpu()[0])
        optimizer.step()
    
    print ('train loss:', np.mean(train_loss))
    print ('train acc:', np.mean(train_acc))
    
    model.eval()
    dev_acc = []
    dev_loss = []
    for sample in validate_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        dev_acc.append(pred_index == index)
        dev_loss.append(loss.data.cpu()[0])
    print ('validation acc:', np.mean(dev_acc))
    print ('validation loss:', np.mean(dev_loss))
    
    model.eval()
    test_acc = []
    test_loss = []
    for sample in test_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        test_acc.append(pred_index == index)
        test_loss.append(loss.data.cpu()[0])
    print ('test acc:', np.mean(test_acc))
    print ('test loss:', np.mean(test_loss))

loading embeddings...
load 8411/8882 embeddings
epoch: 0
train loss: 5.58561
train acc: 0.164926022628
validation acc: 0.161383285303
validation loss: 5.77442
test acc: 0.0862619808307
test loss: 6.10334
epoch: 1
train loss: 3.56558
train acc: 0.293298520453
validation acc: 0.233429394813
validation loss: 4.5549
test acc: 0.145367412141
test loss: 4.62193
epoch: 2
train loss: 2.54174
train acc: 0.434725848564
validation acc: 0.250720461095
validation loss: 4.20967
test acc: 0.143769968051
test loss: 4.30158
epoch: 3
train loss: 2.1061
train acc: 0.506092254134
validation acc: 0.265129682997
validation loss: 4.22048
test acc: 0.166134185304
test loss: 4.19385
epoch: 4
train loss: 1.81055
train acc: 0.54046997389
validation acc: 0.227665706052
validation loss: 4.37767
test acc: 0.158146964856
test loss: 4.24726
epoch: 5
train loss: 1.58914
train acc: 0.58137510879
validation acc: 0.201729106628
validation loss: 4.46867
test acc: 0.174121405751
test loss: 4.34514
epoch: 6
train loss: 1.41

In [24]:
import time
torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

epoch_num = 50
hidden_dim = 200
model = BaselineModel(hidden_dim, 200, len(bioword_dict))
model.cuda()
model.load_embed_bioasq(bioword_dict, './data/vectors.txt', './data/types.txt')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

for epoch in range(epoch_num):
    print ('epoch:', epoch)
    model.train()
    #random.shuffle(train_flat)
    
    train_acc = []
    train_loss = []
    for sample in train_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        
        optimizer.zero_grad()
        pred = model(q, context)
        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        loss.backward()
        
        pred_index = np.argmax(pred[0].data.cpu())
        train_acc.append(pred_index == index)
        train_loss.append(loss.data.cpu()[0])
        optimizer.step()
    
    print ('train loss:', np.mean(train_loss))
    print ('train acc:', np.mean(train_acc))
    
    model.eval()
    dev_acc = []
    dev_loss = []
    for sample in validate_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        dev_acc.append(pred_index == index)
        dev_loss.append(loss.data.cpu()[0])
    print ('validation acc:', np.mean(dev_acc))
    print ('validation loss:', np.mean(dev_loss))
    
    model.eval()
    test_acc = []
    test_loss = []
    for sample in test_flat:
        q = sample.q_words.numpy().tolist()
        context = sample.d_words.numpy().tolist()
        span = sample.span
        index = len(context)*span[0] + span[1]
        pred = model(q, context)

        loss = -torch.log(pred.index_select(1, torch.autograd.Variable(torch.cuda.LongTensor([index]))))
        
        pred_index = np.argmax(pred[0].data.cpu())
        test_acc.append(pred_index == index)
        test_loss.append(loss.data.cpu()[0])
    print ('test acc:', np.mean(test_acc))
    print ('test loss:', np.mean(test_loss))

loading embeddings...
load 8411/8882 embeddings
epoch: 0
train loss: 5.55211
train acc: 0.166666666667
validation acc: 0.164265129683
validation loss: 5.67573
test acc: 0.0958466453674
test loss: 5.97627
epoch: 1
train loss: 3.52896
train acc: 0.299825935596
validation acc: 0.210374639769
validation loss: 4.61779
test acc: 0.153354632588
test loss: 4.61455
epoch: 2
train loss: 2.59761
train acc: 0.43864229765
validation acc: 0.247838616715
validation loss: 4.28466
test acc: 0.148562300319
test loss: 4.32686
epoch: 3
train loss: 2.13134
train acc: 0.495648389904
validation acc: 0.253602305476
validation loss: 4.33734
test acc: 0.14696485623
test loss: 4.25526
epoch: 4
train loss: 1.83969
train acc: 0.541340295909
validation acc: 0.21325648415
validation loss: 4.60204
test acc: 0.148562300319
test loss: 4.43307
epoch: 5
train loss: 1.62996
train acc: 0.569190600522
validation acc: 0.21613832853
validation loss: 4.63278
test acc: 0.174121405751
test loss: 4.38303
epoch: 6
train loss: 1.46

In [ ]:
# extract question-answer-context pair
# lower, tokenization, find span
# question-answer-context -> all to list
# get word dict in bioclean, punctuation dict, assign an embedding to each punctuation
# translate the list token to number including unknown

# train, validation, test, set split
# get question-answer-context-span_list to question-answer-context-span

# evaluation 

In [10]:
print (torch.nn.CrossEntropyLoss()(torch.autograd.Variable(torch.FloatTensor([[1, 2]])), torch.autograd.Variable(torch.LongTensor([0]))))

Variable containing:
 1.3133
[torch.FloatTensor of size 1]



In [56]:
print (len(train))

573


In [57]:
print (len(test))

180


In [55]:
print (len(train_flat))

12897


In [17]:
loss_val.data.cpu().numpy()

array([-0.00093584], dtype=float32)

## model

In [7]:
mini = None
maxi = None
with open('./vectors.txt') as f:
    for i, line in enumerate(f):
        vec = np.array([float(x) for x in line.strip('\n').split()])
        mini = min(vec) if mini is None else min(mini, min(vec))
        maxi = max(vec) if maxi is None else max(maxi, max(vec))

In [8]:
print (mini, maxi)
# CAVEAT: problems in unknown token initialization

-2.25082 3.10778


In [131]:
torch.LongTensor([[[3], [2], [99]], [[4], [1], [88]]]).view(2, -1)


  3   2  99
  4   1  88
[torch.LongTensor of size 2x3]

In [139]:
F.normalize(torch.FloatTensor([[1, 2], [33, 2]]), dim=1, p=1)


 0.3333  0.6667
 0.9429  0.0571
[torch.FloatTensor of size 2x2]

In [169]:
torch.LongTensor(1, 8, 8).random_(0, 25)


(0 ,.,.) = 
   0   6  15  22  14  13  11  22
  18  19   0   1  19  17  12  20
  11  20  11   5  23  24   9   4
   4  15   0  18   2   8   8  10
   0  14   5   9  16  19   4   8
  13  12  15   3  16  18  21   8
  10  19   6   0  18  12  12  14
  13  10  14   2  12  21  14  22
[torch.LongTensor of size 1x8x8]

In [52]:
a = torch.autograd.Variable(torch.FloatTensor([[1, 2, 3]]), requires_grad=True)
print(a)
b = torch.log(a[0][2])

Variable containing:
 1  2  3
[torch.FloatTensor of size 1x3]



In [53]:
b

Variable containing:
 1.0986
[torch.FloatTensor of size 1]

In [51]:
a

Variable containing:
 1
 2
 3
[torch.FloatTensor of size 3]

In [57]:
b

Variable containing:
 1.0986
[torch.FloatTensor of size 1]

In [50]:
b.backward()

In [56]:
b.data.cpu()[0]

1.0986123085021973